<a href="https://colab.research.google.com/github/naderghanbari/mcgill-capstone/blob/master/mcgill_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Collect Montreal firefighter interventions from 2015 to date
%%bash
mkdir -p interventions
curl http://donnees.ville.montreal.qc.ca/dataset/701683f0-a838-4fe0-b4d6-a4964d9ea7f0/resource/9e67d2f3-9104-4af4-aac4-022df621a749/download/donneesouvertes-interventions-sim.csv --silent --output interventions/interventions.csv
echo Number of lines: `wc -l interventions/*`

Number of lines: 640318 interventions/2015-to-2020.csv 640318 interventions/interventions.csv 1280636 total


In [12]:
# Collect Montreal fire stations dataset
%%bash
mkdir -p stations
curl http://donnees.ville.montreal.qc.ca/dataset/c69e78c6-e454-4bd9-9778-e4b0eaf8105b/resource/f6542ad1-31f5-458e-b33d-1a028fab3e98/download/casernes.csv --silent --output stations/stations.csv
echo Number of lines: `wc -l stations/*`

Number of lines: 69 stations/stations.csv


In [9]:
# Collect Montreal crime data from 2015 to date
%%bash
mkdir -p crimes
curl http://donnees.ville.montreal.qc.ca/dataset/5829b5b0-ea6f-476f-be94-bc2b8797769a/resource/c6f482bf-bf0f-4960-8b2f-9982c211addd/download/interventionscitoyendo.csv --silent --output crimes/crimes.csv
echo Number of lines: `wc -l crimes/*`

Number of lines: 171031 crimes/crimes.csv


In [10]:
# Collect Montreal hourly weather data from 2015 to 2019
%%bash
FROM=2015
TO=2019
STATION=30165
mkdir -p weather
for year in $(seq $FROM $TO); do
  for month in {01..12}; do
    curl https://climate.weather.gc.ca/climate_data/bulk_data_e.html\?format\=csv\&stationID\=$STATION\&Year\=$year\&Month\=$month\&Day\=1\&timeframe\=1\&submit\=Download+Data --silent --output weather/$year-$month.csv &
  done;
done

In [11]:
# Count check
import os                                                                                                
assert(len(os.listdir("weather")) == 60)